In [1]:
import pandas as pd
import os
import pickle
import glob
import warnings

In [5]:
ret_sample_path = r'C:\_Files\Personal\Projects\FIAM\FIAM2025\data\ret_sample.csv'

df = pd.read_csv(ret_sample_path)

print(df.columns)

C:\Users\carte\AppData\Local\Temp\ipykernel_29620\2979320123.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ret_sample_path)


Index(['id', 'date', 'ret_eom', 'gvkey', 'iid', 'excntry', 'stock_ret', 'year', 'month', 'char_date',
       ...
       'betadown_252d', 'prc_highprc_252d', 'corr_1260d', 'betabab_1260d', 'rmax5_rvol_21d', 'age', 'qmj', 'qmj_prof', 'qmj_growth', 'qmj_safety'], dtype='object', length=159)


In [2]:
ret_sample_path = r'C:\_Files\Personal\Projects\FIAM\FIAM2025\data\ret_sample.csv'
pd.set_option("display.max_rows", 1000)
df = pd.read_csv(ret_sample_path, usecols=['date'])
print(df.sample(100))

             date
1695191  20101130
5021799  20210831
1109133  20081128
3941632  20180531
2214206  20120831
677865   20070629
3413785  20160831
4202991  20190329
405709   20060721
4193811  20190228
1377171  20091029
3649416  20170630
6369671  20250530
83198    20050531
3856221  20180228
449196   20060929
2960688  20150227
5884210  20240131
4654061  20200731
4325633  20190731
6295073  20250331
3201820  20151230
148835   20050831
2303098  20121228
1256409  20090529
4453515  20191231
1898712  20110729
621200   20070430
4542759  20200331
4155784  20190130
447084   20060929
5997297  20240531
104683   20050630
5108882  20211130
3042563  20150630
1326575  20090831
1114031  20081230
6386094  20250630
2774267  20140731
3925624  20180430
1761546  20110228
1014558  20080829
452082   20060928
1121113  20081230
1415676  20091230
195749   20051031
5188294  20220228
5629329  20230428
2481787  20130731
975004   20080630
2273765  20121129
5051515  20210930
2377526  20130329
4369177  20190930
461199   2

In [8]:
import pandas as pd
import os

def process_csv_in_chunks(input_csv_path, output_dir, chunk_size=1000000):
    """
    Reads and processes a large CSV in chunks to avoid memory errors. It
    uses the 'gvkey' column to save separate CSV files for each unique ID.

    Args:
        input_csv_path (str): The path to the input CSV file.
        output_dir (str): The directory where the output CSVs will be saved.
        chunk_size (int): The number of rows to process in each chunk.
    """
    # Create the output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Created output directory: {output_dir}")

    # This set will keep track of which files we have already written the header for
    created_files = set()
    
    try:
        print(f"Reading data from {input_csv_path} in chunks of {chunk_size} rows...")
        
        # Use the 'chunksize' parameter to create an iterator
        chunk_iterator = pd.read_csv(input_csv_path, low_memory=False, chunksize=chunk_size)
        
        # Loop through each chunk (a smaller DataFrame) from the file
        for i, chunk in enumerate(chunk_iterator):
            print(f"--- Processing Chunk {i+1} ---")

            # --- ID Transformation using 'gvkey' column ---
            print("Cleaning 'gvkey' column and preparing for grouping...")
            # Drop rows where gvkey is missing
            chunk.dropna(subset=['gvkey'], inplace=True)
            # Convert gvkey to integer type to remove decimals (e.g., 1081.0 -> 1081)
            chunk['gvkey'] = chunk['gvkey'].astype(int)
            
            # --- Append rows to the correct files ---
            print("Grouping by 'gvkey' and saving to files...")
            # Group the current chunk by the integer 'gvkey'
            grouped_chunk = chunk.groupby('gvkey')
            
            # Loop through each unique ID found in this chunk
            for current_id, group_df in grouped_chunk:
                file_name = f"{current_id}.csv"
                output_path = os.path.join(output_dir, file_name)
                
                # If we haven't created this file yet, write with a header.
                # Otherwise, append without the header.
                if output_path not in created_files:
                    group_df.to_csv(output_path, index=False, mode='w', header=True)
                    created_files.add(output_path) # Mark this file as created
                else:
                    group_df.to_csv(output_path, index=False, mode='a', header=False)
            
            print(f"Chunk {i+1} processed successfully.")
            # Add a running counter after each chunk is processed
            print(f"Unique IDs found so far: {len(created_files)}")

        print("\nProcessing complete. All chunks have been processed and files saved.")
        # Add a final count at the end of the script
        print(f"Total unique IDs found and files created: {len(created_files)}")

    except FileNotFoundError:
        print(f"Error: The file '{input_csv_path}' was not found. Please check the path.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

# --- How to use this script ---
if __name__ == "__main__":
    print("Starting the memory-efficient script to split data by company ID.")
    
    # 1. Set the path to your input CSV file.
    input_file_path = r'C:\_Files\Personal\Projects\FIAM\FIAM2025\data\ret_sample.csv'
    
    # 2. Set the path for the output folder.
    output_folder_name = r'C:\_Files\Personal\Projects\FIAM\FIAM2025\data\csv_by_id'
    
    # 3. Run the main function.
    process_csv_in_chunks(input_file_path, output_folder_name)



Starting the memory-efficient script to split data by company ID.
Created output directory: C:\_Files\Personal\Projects\FIAM\FIAM2025\data\csv_by_id
Reading data from C:\_Files\Personal\Projects\FIAM\FIAM2025\data\ret_sample.csv in chunks of 1000000 rows...
--- Processing Chunk 1 ---
Cleaning 'gvkey' column and preparing for grouping...
Grouping by 'gvkey' and saving to files...
Chunk 1 processed successfully.
Unique IDs found so far: 29651
--- Processing Chunk 2 ---
Cleaning 'gvkey' column and preparing for grouping...
Grouping by 'gvkey' and saving to files...
Chunk 2 processed successfully.
Unique IDs found so far: 33335
--- Processing Chunk 3 ---
Cleaning 'gvkey' column and preparing for grouping...
Grouping by 'gvkey' and saving to files...
Chunk 3 processed successfully.
Unique IDs found so far: 36931
--- Processing Chunk 4 ---
Cleaning 'gvkey' column and preparing for grouping...
Grouping by 'gvkey' and saving to files...
Chunk 4 processed successfully.
Unique IDs found so far: 

In [3]:
# rf and mgmnt are the risk factors and management discussion and analysis respectively. 
pd.set_option('display.max_columns', None)

# 2. Set display width: This is the key setting to prevent line wrapping.
#    We set it to a very large number to ensure pandas tries to fit everything on one line.
pd.set_option('display.width', 300) # You can increase this number if needed
pd.set_option('display.max_colwidth', 200)

# Optional: If you also want to see more rows in your output
pd.set_option('display.max_rows', 100) # This will show up to 100 rows

directory_path = r'C:\_Files\Personal\Projects\FIAM\FIAM2025\data\text_data'

pickle_pattern = os.path.join(directory_path, '**', '*.pkl')
all_pickle_files = glob.glob(pickle_pattern, recursive=True)

if all_pickle_files:

    first_file_path = all_pickle_files[0]

    df = pd.read_pickle(first_file_path)

    print(df.head)
    print(df.columns)

<bound method NDFrame.head of             date      cik file_type                                                                                                                                                                                                       rf  \
177795  20050103    16099       10Q                                                                                                                                                                                                            
177791  20050103   779544       10K                                                                                                                                                                                                            
177794  20050103   831641       10K                                                                                                                                                                                                            
177790  20

In [1]:
# First, you need to install the 'transformers' library from Hugging Face.
# You can do this by opening your terminal or command prompt and running:
# pip install transformers torch

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

def analyze_sentiment(text_list):
    """
    Analyzes the sentiment of a list of financial texts using a pre-trained FinBERT model.

    Args:
        text_list (list of str): A list of sentences to analyze.

    Returns:
        list of dict: A list of dictionaries, each containing the original text,
                      the predicted sentiment (Positive, Negative, Neutral),
                      and the model's confidence score.
    """
    # 1. Load the Pre-trained FinBERT Model and Tokenizer
    # =======================================================
    # We are using a version of FinBERT that has been specifically fine-tuned for
    # sentiment analysis on financial text.
    model_name = "ProsusAI/finbert"
    print(f"Loading model '{model_name}'... (This may take a moment on first run)")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    print("Model loaded successfully.")

    # 2. Tokenize the Input Text
    # ==========================
    # The tokenizer converts our human-readable sentences into a format of numbers (tokens)
    # that the model can understand. `padding=True` and `truncation=True` handle
    # sentences of different lengths.
    inputs = tokenizer(text_list, padding=True, truncation=True, return_tensors="pt")

    # 3. Get Model Predictions
    # ========================
    # We run the tokenized inputs through the model. `torch.no_grad()` is a performance
    # optimization that tells PyTorch we are not training the model, just using it.
    with torch.no_grad():
        outputs = model(**inputs)

    # 4. Interpret the Results
    # ========================
    # The model's output (logits) are raw scores. We apply a softmax function to
    # convert them into probabilities between 0 and 1.
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    
    # The model was trained on three labels: positive, negative, neutral.
    labels = ["positive", "negative", "neutral"]
    
    results = []
    for i in range(len(text_list)):
        # Find the index of the highest probability score
        sentiment_idx = torch.argmax(predictions[i]).item()
        # Get the corresponding label
        sentiment = labels[sentiment_idx]
        # Get the confidence score for that prediction
        confidence = predictions[i][sentiment_idx].item()
        
        results.append({
            "text": text_list[i],
            "sentiment": sentiment.capitalize(),
            "confidence": f"{confidence:.2%}" # Format as percentage
        })
        
    return results

# --- How to use this script ---
if __name__ == "__main__":
    # Define some example financial sentences to analyze
    example_sentences = [
        "The company's earnings exceeded expectations, driven by strong sales growth.",
        "Regulatory concerns and rising costs have negatively impacted our outlook.",
        "The new factory is expected to begin operations in the third quarter.",
        "Despite a challenging market, the firm maintained its dividend payout.",
        "Profits have plummeted following the product recall."
    ]

    # Get the sentiment analysis results
    sentiment_results = analyze_sentiment(example_sentences)

    # Print the results in a clean format
    print("\n--- FinBERT Sentiment Analysis Results ---")
    for result in sentiment_results:
        print(f"  Sentence:   '{result['text']}'")
        print(f"  Sentiment:  {result['sentiment']} (Confidence: {result['confidence']})\n")


c:\_Files\Personal\Projects\FIAM\FIAM2025\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading model 'ProsusAI/finbert'... (This may take a moment on first run)


c:\_Files\Personal\Projects\FIAM\FIAM2025\myenv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\carte\.cache\huggingface\hub\models--ProsusAI--finbert. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular

Model loaded successfully.

--- FinBERT Sentiment Analysis Results ---
  Sentence:   'The company's earnings exceeded expectations, driven by strong sales growth.'
  Sentiment:  Positive (Confidence: 95.96%)

  Sentence:   'Regulatory concerns and rising costs have negatively impacted our outlook.'
  Sentiment:  Negative (Confidence: 94.92%)

  Sentence:   'The new factory is expected to begin operations in the third quarter.'
  Sentiment:  Neutral (Confidence: 87.01%)

  Sentence:   'Despite a challenging market, the firm maintained its dividend payout.'
  Sentiment:  Positive (Confidence: 95.12%)

  Sentence:   'Profits have plummeted following the product recall.'
  Sentiment:  Negative (Confidence: 97.40%)



Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [4]:
import os
import glob
import pickle
import pandas as pd # Import pandas to handle the DataFrame

def inspect_first_filing(data_directory, output_filename="filing_example.txt"):
    """
    Finds the first pickle file, searches for the first row with non-empty text
    sections, and saves that filing's text to a file for inspection.

    Args:
        data_directory (str): The root path where the text data is stored.
        output_filename (str): The name of the text file to create.
    """
    print(f"Searching for pickle files in '{data_directory}'...")
    
    # 1. Find the first pickle file automatically
    # ============================================
    pickle_pattern = os.path.join(data_directory, '**', '*.pkl')
    all_pickle_files = glob.glob(pickle_pattern, recursive=True)

    if not all_pickle_files:
        print("Error: No pickle files were found. Please check the directory path.")
        return

    first_file_path = all_pickle_files[0]
    print(f"Found files. Loading the first one: '{os.path.basename(first_file_path)}'")

    # 2. Load the data from the pickle file
    # =======================================
    try:
        with open(first_file_path, 'rb') as f:
            filings_df = pickle.load(f)
            
            if filings_df.empty:
                print("The pickle file contains an empty DataFrame.")
                return

            # --- NEW: Loop to find the first valid row ---
            # We'll search for the first row that has meaningful content in both key sections.
            valid_filing_series = None
            for index, row in filings_df.iterrows():
                # The correct column names are 'rf' (Risk Factors) and 'mgmt' (MD&A)
                risk_text = row.get('rf')
                mda_text = row.get('mgmt')

                # Check if both sections exist and have a reasonable length (e.g., > 100 characters)
                if pd.notna(risk_text) and len(str(risk_text).strip()) > 100 and \
                   pd.notna(mda_text) and len(str(mda_text).strip()) > 100:
                    valid_filing_series = row
                    print(f"Found a suitable filing for inspection at index {index}.")
                    break # Exit the loop once a good example is found
            
            if valid_filing_series is None:
                print("Error: Could not find a single filing with complete text sections in this file.")
                return

            # 3. Extract data from the VALID filing row
            # ==========================================================
            cik = valid_filing_series.get('cik', 'CIK not found')
            filing_date = valid_filing_series.get('date', 'Date not found')
            risk_factors_text = valid_filing_series.get('rf', 'Section not found.')
            mda_text = valid_filing_series.get('mgmt', 'Section not found.')

            print(f"Extracting data for CIK: {cik}, Filing Date: {filing_date}")


    except Exception as e:
        print(f"An error occurred while loading or processing the pickle file: {e}")
        return

    # 4. Write the extracted text to a plain text file
    # =================================================
    try:
        with open(output_filename, 'w', encoding='utf-8') as f:
            f.write("="*80 + "\n")
            f.write(f"EXAMPLE FILING FOR INSPECTION\n")
            f.write(f"CIK: {cik} | DATE: {filing_date}\n")
            f.write("="*80 + "\n\n")

            f.write("--- RISK FACTORS ---\n\n")
            f.write(str(risk_factors_text)) # Use str() to handle potential non-string data
            f.write("\n\n")

            f.write("="*80 + "\n\n")

            f.write("--- MANAGEMENT'S DISCUSSION AND ANALYSIS (MD&A) ---\n\n")
            f.write(str(mda_text)) # Use str() to handle potential non-string data
        
        print(f"\nSuccess! The content has been written to '{output_filename}'")
        print("You can now open this file to see the raw text.")

    except Exception as e:
        print(f"An error occurred while writing the file: {e}")


# --- How to use this script ---
if __name__ == "__main__":
    # !! IMPORTANT: Make sure this path points to your unzipped text data directory !!
    text_data_path = r'C:\_Files\Personal\Projects\FIAM\FIAM2025\data\text_data'
    
    inspect_first_filing(text_data_path)



Searching for pickle files in 'C:\_Files\Personal\Projects\FIAM\FIAM2025\data\text_data'...
Found files. Loading the first one: 'text_us_2005.pkl'


C:\Users\carte\AppData\Local\Temp\ipykernel_19912\3822597473.py:33: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  filings_df = pickle.load(f)


Found a suitable filing for inspection at index 179123.
Extracting data for CIK: 879526, Filing Date: 20050303

Success! The content has been written to 'filing_example.txt'
You can now open this file to see the raw text.


In [12]:
import os
import glob
import pickle
import pandas as pd
import re # Import the regular expression module

def is_title(line):
    """
    Heuristic to determine if a line of text is a title.
    A line is considered a title if it's all uppercase, doesn't end with a period,
    and has a reasonable length.
    """
    # Increased the word count limit to 35 to catch longer titles.
    return line.isupper() and len(line.split()) < 35 and not line.strip().endswith('.') and len(line) > 5

def parse_risk_factors(raw_text):
    """
    Parses the raw text of a 'Risk Factors' section to intelligently combine titles
    with their corresponding paragraphs. This version is more robust.

    Args:
        raw_text (str): The full text of the risk factors section.

    Returns:
        str: A formatted string with titles reliably merged with their paragraphs.
    """
    if not isinstance(raw_text, str):
        return "Invalid text input."

    # Split the entire text block into paragraphs based on one or more empty lines.
    raw_paragraphs = re.split(r'\n\s*\n', raw_text.strip())
    
    # Clean up whitespace, join any internally wrapped lines, and filter out page numbers.
    clean_paragraphs = []
    for para in raw_paragraphs:
        clean_para = ' '.join(para.split())
        # Add a check to filter out paragraphs that are just numbers (page numbers)
        if clean_para and not clean_para.isdigit():
            clean_paragraphs.append(clean_para)

    # --- New, More Robust Merging Logic ---
    parsed_chunks = []
    held_title = ""
    for para in clean_paragraphs:
        if is_title(para):
            # If we were already holding a title, it means we found two titles in a row.
            # The previous one must have been a standalone title or sub-header.
            if held_title:
                parsed_chunks.append(held_title)
            # Now, hold the new title, waiting for its paragraph.
            held_title = para
        else: # This is a regular content paragraph.
            # If we have a title waiting, prepend it to this paragraph and reset.
            if held_title:
                parsed_chunks.append(f"{held_title}\n{para}")
                held_title = "" # Clear the held title as it has been used
            else:
                # This is just a regular paragraph with no title immediately before it.
                parsed_chunks.append(para)

    # After the loop, if there's a title left over (i.e., the very last item was a title),
    # make sure to add it to the final list.
    if held_title:
        parsed_chunks.append(held_title)
    print(parsed_chunks)  
    return "\n\n".join(parsed_chunks)


def inspect_first_filing(data_directory, output_filename="filing_example_titles.txt"):
    """
    Finds the first pickle file, searches for the first row with non-empty text
    sections, and saves that filing's text to a file for inspection.
    """
    print(f"Searching for pickle files in '{data_directory}'...")
    
    # 1. Find the first pickle file automatically
    # ============================================
    pickle_pattern = os.path.join(data_directory, '**', '*.pkl')
    all_pickle_files = glob.glob(pickle_pattern, recursive=True)

    if not all_pickle_files:
        print("Error: No pickle files were found. Please check the directory path.")
        return

    first_file_path = all_pickle_files[0]
    print(f"Found files. Loading the first one: '{os.path.basename(first_file_path)}'")

    # 2. Load the data from the pickle file
    # =======================================
    try:
        with open(first_file_path, 'rb') as f:
            filings_df = pickle.load(f)
            
            if filings_df.empty:
                print("The pickle file contains an empty DataFrame.")
                return

            valid_filing_series = None
            for index, row in filings_df.iterrows():
                risk_text = row.get('rf')
                mda_text = row.get('mgmt')

                if pd.notna(risk_text) and len(str(risk_text).strip()) > 100 and \
                   pd.notna(mda_text) and len(str(mda_text).strip()) > 100:
                    valid_filing_series = row
                    print(f"Found a suitable filing for inspection at index {index}.")
                    break
            
            if valid_filing_series is None:
                print("Error: Could not find a single filing with complete text sections in this file.")
                return

            # 3. Extract data from the VALID filing row
            # ==========================================================
            cik = valid_filing_series.get('cik', 'CIK not found')
            filing_date = valid_filing_series.get('date', 'Date not found')
            risk_factors_text_raw = valid_filing_series.get('rf', 'Section not found.')
            mda_text = valid_filing_series.get('mgmt', 'Section not found.')
            
            # --- Use the NEW, more robust parsing function ---
            risk_factors_text_parsed = parse_risk_factors(risk_factors_text_raw)

            print(f"Extracting data for CIK: {cik}, Filing Date: {filing_date}")


    except Exception as e:
        print(f"An error occurred while loading or processing the pickle file: {e}")
        return

    # 4. Write the extracted text to a plain text file
    # =================================================
    try:
        with open(output_filename, 'w', encoding='utf-8') as f:
            f.write("="*80 + "\n")
            f.write(f"EXAMPLE FILING FOR INSPECTION\n")
            f.write(f"CIK: {cik} | DATE: {filing_date}\n")
            f.write("="*80 + "\n\n")

            f.write("--- RISK FACTORS (PARSED) ---\n\n")
            f.write(str(risk_factors_text_parsed)) # Write the newly parsed text
            f.write("\n\n")

            f.write("="*80 + "\n\n")

            f.write("--- MANAGEMENT'S DISCUSSION AND ANALYSIS (MD&A) ---\n\n")
            f.write(str(mda_text))
        
        print(f"\nSuccess! The content has been written to '{output_filename}'")
        print("You can now open this file to see the raw text with parsed risk factors.")

    except Exception as e:
        print(f"An error occurred while writing the file: {e}")


# --- How to use this script ---
if __name__ == "__main__":
    # !! IMPORTANT: Make sure this path points to your unzipped text data directory !!
    text_data_path = r'C:\_Files\Personal\Projects\FIAM\FIAM2025\data\text_data'
    
    inspect_first_filing(text_data_path)



Searching for pickle files in 'C:\_Files\Personal\Projects\FIAM\FIAM2025\data\text_data'...
Found files. Loading the first one: 'text_us_2005.pkl'


C:\Users\carte\AppData\Local\Temp\ipykernel_19912\383267985.py:92: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  filings_df = pickle.load(f)


Found a suitable filing for inspection at index 179123.
['Item 7B Risk Factors.................................................................... 24 ITEM 7B - RISK FACTORS', 'You should carefully consider the risks described below in addition to other information contained or incorporated by reference in this Report before investing in our securities. Realization of any of the following risks could have a material adverse effect on our business, financial condition, cash flows and results of operations.', 'RISKS RELATED TO OUR BUSINESS, STRATEGY AND OPERATIONS\nWE HAVE GENERATED SIGNIFICANT LOSSES IN RECENT PERIODS.', 'We incurred significant net losses during the three years prior to 2004. In 2004, we reported net income of 58.4 million, but we reported net losses of 232.2 million, 56.2 million and 57.2 million for the years ended December 31, 2001, 2002 and 2003, respectively. Our ability to achieve and sustain profitability in the future will depend on the successful continued impl

In [14]:
import os
import glob
import pickle
import pandas as pd

def inspect_multiple_filings(data_directory, num_to_inspect=10, output_filename="filing_examples_raw.txt"):
    """
    Finds and processes a specified number of valid filings and saves their raw,
    unformatted risk factor sections to a single text file for comparison.
    """
    print(f"Searching for pickle files in '{data_directory}'...")
    
    pickle_pattern = os.path.join(data_directory, '**', '*.pkl')
    all_pickle_files = glob.glob(pickle_pattern, recursive=True)

    if not all_pickle_files:
        print("Error: No pickle files were found. Please check the directory path.")
        return

    valid_filings_found = []
    
    # Loop through the pickle files until we find enough examples
    for file_path in all_pickle_files:
        if len(valid_filings_found) >= num_to_inspect:
            break

        print(f"Loading file: '{os.path.basename(file_path)}'")
        try:
            with open(file_path, 'rb') as f:
                filings_df = pickle.load(f)
            
            if filings_df.empty:
                continue # Skip empty dataframes

            # Loop through rows to find filings with complete text
            for index, row in filings_df.iterrows():
                risk_text = row.get('rf')
                mda_text = row.get('mgmt')

                if pd.notna(risk_text) and len(str(risk_text).strip()) > 100 and \
                   pd.notna(mda_text) and len(str(mda_text).strip()) > 100:
                    
                    valid_filings_found.append(row)
                    print(f"  > Found valid filing #{len(valid_filings_found)} at index {index}.")
                    
                    if len(valid_filings_found) >= num_to_inspect:
                        break # Stop searching rows if we have enough
        
        except Exception as e:
            print(f"An error occurred while processing {os.path.basename(file_path)}: {e}")

    if not valid_filings_found:
        print("Could not find any valid filings to inspect.")
        return

    # Now, write all found examples to a single file
    print(f"\nFound {len(valid_filings_found)} valid filings. Writing to '{output_filename}'...")
    try:
        with open(output_filename, 'w', encoding='utf-8') as f:
            for i, filing_series in enumerate(valid_filings_found):
                cik = filing_series.get('cik', 'N/A')
                filing_date = filing_series.get('date', 'N/A')
                # --- MODIFICATION: Use the raw text directly ---
                risk_factors_raw = filing_series.get('rf', '')
                
                f.write("="*80 + "\n")
                f.write(f"--- EXAMPLE {i+1} of {len(valid_filings_found)} ---\n")
                f.write(f"CIK: {cik} | DATE: {filing_date}\n")
                f.write("="*80 + "\n\n")
                
                # Write the raw, unparsed text to the file
                f.write(str(risk_factors_raw))
                f.write("\n\n\n")

        print("Success! The file has been created with raw, unformatted text.")
    except Exception as e:
        print(f"An error occurred while writing the file: {e}")


# --- How to use this script ---
if __name__ == "__main__":
    # !! IMPORTANT: Make sure this path points to your unzipped text data directory !!
    text_data_path = r'C:\_Files\Personal\Projects\FIAM\FIAM2025\data\text_data'
    
    inspect_multiple_filings(text_data_path, num_to_inspect=10)



Searching for pickle files in 'C:\_Files\Personal\Projects\FIAM\FIAM2025\data\text_data'...
Loading file: 'text_us_2005.pkl'


C:\Users\carte\AppData\Local\Temp\ipykernel_19912\708864471.py:30: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  filings_df = pickle.load(f)


  > Found valid filing #1 at index 179123.
  > Found valid filing #2 at index 179477.
  > Found valid filing #3 at index 179927.
  > Found valid filing #4 at index 180158.
  > Found valid filing #5 at index 180050.
  > Found valid filing #6 at index 180115.
  > Found valid filing #7 at index 180558.
  > Found valid filing #8 at index 180635.
  > Found valid filing #9 at index 180394.
  > Found valid filing #10 at index 181003.

Found 10 valid filings. Writing to 'filing_examples_raw.txt'...
Success! The file has been created with raw, unformatted text.


In [21]:
import os
import glob
import pickle
import pandas as pd
import re # Import the regular expression module

def is_header_heuristic(chunk):
    """
    A new, more precise heuristic based on user findings. A chunk is a header if:
    1. It is a single sentence.
    2. It starts with a capital letter and ends with a period.
    3. It is relatively short (under 25 words).
    This avoids flagging longer, single-sentence content paragraphs as headers.
    """
    if not isinstance(chunk, str):
        return False
        
    stripped_chunk = chunk.strip()

    # Rule 1 & 2: Must be a single sentence starting with a cap and ending with a period.
    # We check if there's exactly one period and it's at the end.
    is_single_sentence = stripped_chunk.count('.') == 1 and stripped_chunk.endswith('.')
    if not is_single_sentence:
        return False

    # Check that it starts with a capital letter.
    if not stripped_chunk[0].isupper():
        return False
        
    # Rule 3: Must be relatively short to be considered a title.
    is_short = len(stripped_chunk.split()) < 40
    if not is_short:
        return False

    # If it passes all checks, it's very likely a title.
    return True

def clean_and_structure_risks(raw_text):
    """
    Intelligently parses raw risk factor text and returns a LIST of structured
    risk items, where each item is a header plus its content.
    """
    if not isinstance(raw_text, str):
        return []

    # 1. Split text into logical chunks based on empty lines.
    chunks = re.split(r'\n\s*\n', raw_text.strip())
    
    # 2. Clean up each chunk and filter out noise.
    cleaned_chunks = []
    for chunk in chunks:
        processed_chunk = ' '.join(chunk.split())
        if processed_chunk and not processed_chunk.isdigit():
            cleaned_chunks.append(processed_chunk)

    # 3. Group chunks into structured risk items (header + content).
    if not cleaned_chunks:
        return []

    structured_items = []
    current_item_chunks = []

    for chunk in cleaned_chunks:
        if is_header_heuristic(chunk):
            if current_item_chunks:
                structured_items.append("\n".join(current_item_chunks))
            current_item_chunks = [chunk]
        else:
            if not current_item_chunks:
                current_item_chunks = [chunk]
            else:
                current_item_chunks.append(chunk)
    
    if current_item_chunks:
        structured_items.append("\n".join(current_item_chunks))

    return structured_items


def inspect_multiple_filings(data_directory, num_to_inspect=10, output_filename="filing_examples_parsed.txt"):
    """
    Finds and processes a specified number of valid filings and saves their
    intelligently parsed risk factor sections to a single text file.
    """
    print(f"Searching for pickle files in '{data_directory}'...")
    
    pickle_pattern = os.path.join(data_directory, '**', '*.pkl')
    all_pickle_files = glob.glob(pickle_pattern, recursive=True)

    if not all_pickle_files:
        print("Error: No pickle files were found. Please check the directory path.")
        return

    valid_filings_found = []
    
    for file_path in all_pickle_files:
        if len(valid_filings_found) >= num_to_inspect: break
        print(f"Loading file: '{os.path.basename(file_path)}'")
        try:
            with open(file_path, 'rb') as f: filings_df = pickle.load(f)
            if filings_df.empty: continue

            for index, row in filings_df.iterrows():
                risk_text = row.get('rf')
                mda_text = row.get('mgmt')

                if pd.notna(risk_text) and len(str(risk_text).strip()) > 100 and \
                   pd.notna(mda_text) and len(str(mda_text).strip()) > 100:
                    valid_filings_found.append(row)
                    print(f"  > Found valid filing #{len(valid_filings_found)} at index {index}.")
                    if len(valid_filings_found) >= num_to_inspect: break
        
        except Exception as e:
            print(f"An error occurred while processing {os.path.basename(file_path)}: {e}")

    if not valid_filings_found:
        print("Could not find any valid filings to inspect.")
        return

    print(f"\nFound {len(valid_filings_found)} valid filings. Writing to '{output_filename}'...")
    try:
        with open(output_filename, 'w', encoding='utf-8') as f:
            for i, filing_series in enumerate(valid_filings_found):
                cik = filing_series.get('cik', 'N/A')
                filing_date = filing_series.get('date', 'N/A')
                risk_factors_raw = filing_series.get('rf', '')
                
                # This function now returns a list of strings
                risk_items_list = clean_and_structure_risks(risk_factors_raw)
                
                f.write("="*80 + "\n")
                f.write(f"--- EXAMPLE FILING {i+1} of {len(valid_filings_found)} ---\n")
                f.write(f"CIK: {cik} | DATE: {filing_date}\n")
                f.write("="*80 + "\n\n")
                
                if not risk_items_list:
                    f.write("--> No risk chunks could be parsed for this filing.\n\n")
                else:
                    for j, risk_item in enumerate(risk_items_list):
                        f.write(f"---------- Risk Chunk {j+1} ----------\n")
                        f.write(risk_item)
                        f.write("\n\n") # Add extra space after each chunk
                
                f.write("\n\n") # Extra space between filings

        print("Success! The file has been created with clear separators for each risk chunk.")
    except Exception as e:
        print(f"An error occurred while writing the file: {e}")


# --- How to use this script ---
if __name__ == "__main__":
    text_data_path = r'C:\_Files\Personal\Projects\FIAM\FIAM2025\data\text_data'
    inspect_multiple_filings(text_data_path, num_to_inspect=10)



Searching for pickle files in 'C:\_Files\Personal\Projects\FIAM\FIAM2025\data\text_data'...
Loading file: 'text_us_2005.pkl'


C:\Users\carte\AppData\Local\Temp\ipykernel_19912\1484652006.py:100: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  with open(file_path, 'rb') as f: filings_df = pickle.load(f)


  > Found valid filing #1 at index 179123.
  > Found valid filing #2 at index 179477.
  > Found valid filing #3 at index 179927.
  > Found valid filing #4 at index 180158.
  > Found valid filing #5 at index 180050.
  > Found valid filing #6 at index 180115.
  > Found valid filing #7 at index 180558.
  > Found valid filing #8 at index 180635.
  > Found valid filing #9 at index 180394.
  > Found valid filing #10 at index 181003.

Found 10 valid filings. Writing to 'filing_examples_parsed.txt'...
Success! The file has been created with clear separators for each risk chunk.


In [7]:
import torch
print(torch.version.cuda)

None
